<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/ANN_regression/soft_sensor_debutanizer_part_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set 28 2023

In [2]:
# parte 1: leitura de dados
#  np.loadtxt foi o mais adequado para leitura

In [3]:
import pandas as pd
import numpy as np

In [4]:
!git clone https://github.com/ucfilho/Machine_Learning_DataSet
%cd Machine_Learning_DataSet

Cloning into 'Machine_Learning_DataSet'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 231 (delta 12), reused 0 (delta 0), pack-reused 195
Receiving objects: 100% (231/231), 67.75 MiB | 46.72 MiB/s, done.
Resolving deltas: 100% (76/76), done.
/content/Machine_Learning_DataSet


In [5]:
data = np.loadtxt('debutanizer_data.txt')

In [6]:
X = data[:,:7]
y = data[:,7]

In [7]:
X.shape,y.shape

((2394, 7), (2394,))

In [8]:
# parte 2: divisao de dados
#  versao tranquila
#  falta ver o q representa y1 e y2, fiz aqui so com y1 para comecar a historia
#   colocar o valor fixo do test_size conforme os artigos (anotar)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# se for serie temporar tem q fazer diferente

In [11]:
# Parte 3: modelo -trocar ML por varios, fiz somente um

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [13]:
model = RandomForestRegressor( random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [14]:
r2_test= r2_score(y_test,y_pred)
print('r2 (test)=',r2_test)

r2 (test)= 0.8035237326250433


In [ ]:
# parte 04: otimizar o metodo
# utilizei randomizedSearchCV mas nao melhorou muito em relacao ao default
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [15]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [18]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [19]:
rf_random.best_params_

{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}

In [23]:
best_random = rf_random.best_estimator_
predictions = best_random.predict(X_test)
r2_test= r2_score(y_test,predictions)
print('r2 (test) best model=',r2_test)

r2 (test) best model= 0.8037884762737995
